In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import os
import json
import math

In [ ]:
def load_solomon_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        if "VEHICLE" in line:
            vehicle_info_line = lines[i + 2]
            break

    vehicle_number_line = vehicle_info_line.split()
    vehicle_number = int(vehicle_number_line[0])
    vehicle_capacity = int(vehicle_number_line[1])
    vehicle_capacitiesLB = [vehicle_capacity] * vehicle_number

    demandLB = []
    time_windows = []
    coords = []
    due_time_0 = 0

    for line in lines[i + 4:]:
        parts = line.split()
        if len(parts) == 7:
            customer_id = int(parts[0])
            x_coord = int(parts[1])
            y_coord = int(parts[2])
            demand = int(parts[3])
            ready_time = int(parts[4])
            due_time = int(parts[5])
            service_time = int(parts[6])

            if customer_id == 0:
                due_time_0 = due_time
            demandLB.append(demand)
            time_windows.append((ready_time, due_time))
            coords.append((x_coord, y_coord))

    print(due_time_0)
    print(service_time)

    return vehicle_number, vehicle_capacitiesLB, demandLB, time_windows, coords, service_time, due_time_0

# int

In [ ]:
def create_data_model(file_path):
    data = {}
    vehicle_number, vehicle_capacitiesLB, demandLB, time_windows, coords, service_time, due_time_0 = load_solomon_data(file_path)

    data["demands"] = demandLB
    data["vehicle_capacities"] = vehicle_capacitiesLB
    data["num_vehicles"] = vehicle_number
    data["depot"] = 0 
    data["time_windows"] = time_windows

    def euclidean_distance(x1, y1, x2, y2):
        return int(((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5)

    distance_matrix = [
        [0 if i == j else euclidean_distance(coords[i][0], coords[i][1], coords[j][0], coords[j][1]) for j in range(len(coords))]
        for i in range(len(coords))
    ]
    data["distance_matrix"] = distance_matrix
    print(data["distance_matrix"])
    return data, service_time, due_time_0

In [ ]:
def print_solution(data, manager, routing, solution):
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    total_distance = 0
    total_load = 0

    all_routes = []
    
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        route_nodes = [] 
        
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_nodes.append(node_index) 
            route_load += data["demands"][node_index]
            time_var = time_dimension.CumulVar(index)
            plan_output += f"{node_index} Load({route_load}) Time({solution.Min(time_var)},{solution.Max(time_var)}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        route_nodes.append(manager.IndexToNode(index)) 
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        
        total_distance += route_distance
        total_load += route_load
        total_time += solution.Min(time_var)
        
        all_routes.append(route_nodes)

    all_routes = [route for route in all_routes if len(route) > 2]

    print("Routes of all vehicles (valid routes):")
    print(all_routes)

    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")
    print(f"Total time of all routes: {total_time}min")
    return all_routes, total_distance

In [ ]:
def main(file_link, run_time, first_solution_strategy="PATH_CHEAPEST_ARC"):
    file_path = f'{file_link}'
    data, service_time, due_time_0 = create_data_model(file_path)

    manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]), data["num_vehicles"], data["depot"])
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, data["vehicle_capacities"], True, "Capacity")

    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]  # Assume time is proportional to distance

    time_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.AddDimension(time_callback_index, service_time, due_time_0, False, "Time") #90, 1236

    time_dimension = routing.GetDimensionOrDie("Time")
    for location_idx, time_window in enumerate(data["time_windows"]):
        if location_idx == data["depot"]:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

    # for location_idx, time_window in enumerate(data["time_windows"]):
    #     if location_idx == data["depot"]:
    #         continue
    #     index = manager.NodeToIndex(location_idx)
    #     time_var = time_dimension.CumulVar(index)
        
    #     # Debugging: Print the time variable and index
    #     print(f"Setting time window for location {location_idx} (Index: {index})")
    #     print(f"Time window: {time_window}")
        
    #     # Ensure the time variable is valid
    #     if time_var is not None:
    #         time_var.SetRange(time_window[0], time_window[1])
    #     else:
    #         print(f"Error: Time variable for location {location_idx} is None!")


    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(data["time_windows"][data["depot"]][0], data["time_windows"][data["depot"]][1])

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    # search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC #PATH_CHEAPEST_ARC GLOBAL_CHEAPEST_ARC LOCAL_CHEAPEST_INSERTION BEST_INSERTION ALL_UNPERFORMED

    if first_solution_strategy == "PATH_CHEAPEST_ARC":
        search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    elif first_solution_strategy == "GLOBAL_CHEAPEST_ARC":
        search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.GLOBAL_CHEAPEST_ARC
    elif first_solution_strategy == "LOCAL_CHEAPEST_INSERTION":
        search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_INSERTION
    else:
        search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC  # Default value

    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH #GUIDED_LOCAL_SEARCH REACTIVE_TABU_SEARCH SIMULATED_ANNEALING TABU_SEARCH
    search_parameters.time_limit.FromSeconds(run_time)  

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        all_routes, total_distance = print_solution(data, manager, routing, solution)
    
    return all_routes, total_distance

In [ ]:
# file_folder = ["c1", "c2", "homberger_200_customer_instances", "homberger_400_customer_instances", 
#                "homberger_800_customer_instances", "homberger_1000_customer_instances", 
#                "r1", "r2", "rc1", "rc2"]
# file_folder = ["r1", "r2", "rc1", "rc2"]
# # file_folder = ["c0"]
# time = [5, 30, 100, 300, 600, 900, 1200]

# base_path = "/home/wsl/CVRPTW/data"
# save_path = "/home/wsl/CVRPTW/data/routes_distance"
# result_path = "/home/wsl/CVRPTW/data/result"

file_folder = ["c109"]
# file_folder = ["c0"]
time = [5, 200, 300, 600]

base_path = "/home/wsl/CVRPTW/data"
save_path = "/home/wsl/CVRPTW/data/c109"
result_path = "/home/wsl/CVRPTW/data/c109"

all_conversations = []

for folder in file_folder:
    folder_path = os.path.join(base_path, folder)
    try:
        files = os.listdir(folder_path)  
        for file_name in files:
            file_link = os.path.join(folder_path, file_name)
            with open(file_link, 'r') as file:
                file_content = file.read() 

            conversation = {
                "conversations": [],
                "system": (
                    "xx"
                )
            }
            
            i = 0
            distance_list = []
            pre_routes = []
            file_results = [] 

            for run_time in time:

                all_routes, total_distance = main(file_link, run_time)

                if i>=2 and distance_list and total_distance == distance_list[i-1] and total_distance == distance_list[i-2]:
                    print(f"Total distance for run_time {run_time} is the same as the previous round. Skipping to the next file.")
                    break
                
                # if not distance_list:
                #     distance_list.append(total_distance)
                # elif distance_list and run_time < 400:
                #     if all_routes == pre_routes:
                #         all_routes, total_distance = main(file_link, run_time, "GLOBAL_CHEAPEST_ARC")
                #         if all_routes == pre_routes or all_routes is None:
                #             all_routes, total_distance = main(file_link, run_time, "LOCAL_CHEAPEST_INSERTION")
                #             if all_routes is None:
                #                 all_routes = pre_routes
                #                 total_distance = distance_list[i-1]
                #     distance_list.append(total_distance)
                # else:
                #     # 其他情况下也执行distance_list.append(total_distance)
                #     distance_list.append(total_distance)
 
                distance_list.append(total_distance)

                gpt_value = json.dumps(all_routes)

                if run_time == 5:
                    human_value = f"xx"
                else:
                    human_value = f"xx"

                conversation["conversations"].append({
                    "from": "human",
                    "value": human_value
                })

                
                conversation["conversations"].append({
                    "from": "gpt",
                    "value": gpt_value 
                })
                
                pre_routes = all_routes
                i += 1

                file_results.append({
                    "run_time": run_time,
                    "all_routes": all_routes,
                    "total_distance": total_distance
                })
            
            result_file_path = os.path.join(save_path, f"{file_name}_results.txt")
            with open(result_file_path, 'w') as result_file:
                for result in file_results:
                    result_file.write(f"Run Time: {result['run_time']}\n")
                    result_file.write(f"All Routes: {json.dumps(result['all_routes'])}\n")
                    result_file.write(f"Total Distance: {result['total_distance']}\n\n")

            output_json_path = os.path.join(result_path, f"conversations_output_{file_name}.json")
            with open(output_json_path, 'w') as json_file:
                json.dump(conversation, json_file, indent=4)

            print(f"Conversation for {file_name} has been saved to '{output_json_path}'")

            print(f"this All_conversation is {conversation}")
            all_conversations.append(conversation)

    except FileNotFoundError:
        print(f"file folder {folder} not found")

# print(f"this All_conversation is {all_conversations}")
# print(" ")

try:
    with open('/home/wsl/CVRPTW/conversations_output.json', 'w') as json_file:
        json.dump(all_conversations, json_file, indent=4)
    print("All conversations have been saved to 'conversations_output.json'")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
file_folder = ["test_data_1"]
time = [5, 100]

base_path = "/home/wsl/CVRPTW/data"
save_path = "/home/wsl/CVRPTW/data/test_data_1"
result_path = "/home/wsl/CVRPTW/data/test_data_1"

all_conversations = []

for folder in file_folder:
    folder_path = os.path.join(base_path, folder)
    try:
        files = os.listdir(folder_path)  
        for file_name in files:
            file_link = os.path.join(folder_path, file_name)
            print(file_name)
            with open(file_link, 'r') as file:
                file_content = file.read() 
            
            i = 0
            distance_list = []
            pre_routes = []
            file_results = [] 
            for run_time in time:
                all_routes, total_distance = main(file_link, run_time)

                if i>=2 and distance_list and total_distance == distance_list[i-1] and total_distance == distance_list[i-2]:
                    print(f"Total distance for run_time {run_time} is the same as the previous round. Skipping to the next file.")
                    break

                distance_list.append(total_distance)
                
                pre_routes = all_routes
                i += 1

                file_results.append({
                    "run_time": run_time,
                    "all_routes": all_routes,
                    "total_distance": total_distance
                })
            
            result_file_path = os.path.join(save_path, f"{file_name}_results.txt")
            with open(result_file_path, 'w') as result_file:
                for result in file_results:
                    result_file.write(f"Run Time: {result['run_time']}\n")
                    result_file.write(f"All Routes: {json.dumps(result['all_routes'])}\n")
                    result_file.write(f"Total Distance: {result['total_distance']}\n\n")

    except FileNotFoundError:
        print(f"File folder {folder} not found")
